In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LogNorm
from matplotlib.ticker import LogFormatterExponent
from DTA.enrichment_plotters import polar_plot, get_file_list, get_helices, read_rep
from DTA.polarDensity_helper import Coord_Get, get_header_info
from DTA.site_distributions import outline_site, make_symmetric_sites, combine_sites, plot_density,make_simple_site, Site, plot_bulk_counts
plt.rcParams['axes.grid'] = False 

In [ ]:
# Enrichment plot colormap
from matplotlib.colors import ListedColormap
depleted = plt.cm.get_cmap('RdGy_r', 256)
enriched = plt.cm.get_cmap('bwr', 256)
newcolors = np.concatenate([depleted(np.linspace(0.35, 0.5, 128)), enriched(np.linspace(0.5,1,128))])
my_cmap = ListedColormap(newcolors)

# Read and plot the distribution
In VMD:
1. load the trajectory of the empty membrane
2. modify do_get_counts.tcl to have the total area of the site
3. source do_get_counts.tcl
4. It will save to a file called counts_[area].out
5. Provide the path here:

In [ ]:
area = 50
root = Path(f"../accessible_area/")
bulk_counts_path = root.joinpath(f"POPG_counts_{area}.dat")
bulk_counts = np.loadtxt(bulk_counts_path)

frequencies = np.bincount(bulk_counts.astype(int).flatten())
probabilities = frequencies/np.sum(frequencies)
fig, ax = plt.subplots()
ax.plot(range(len(probabilities)), probabilities)
ax.set_ylabel("Probability")
ax.set_xlabel(f"Number of beads in an area about {area} "+r"$\AA^2$")
bulk_mode = np.argmax(probabilities)
ax.vlines([bulk_mode], 0, np.max(probabilities), color = 'black', linestyles='dashed', label = f"mode={bulk_mode}")
ax.legend()

In [ ]:
probabilities[0]

In [ ]:

fig, axes = plt.subplots(2, 1, figsize=(3,6), sharex=True, sharey=True)

area = 50
#root = Path(f"/Users/ezry/Projects/Polar_Binning_DeltaG/accessible_area/")
bulk_counts_path = root.joinpath(f"POPG_counts_{area}.dat")
bulk_counts = np.loadtxt(bulk_counts_path)
frequencies = np.bincount(bulk_counts.astype(int).flatten())
probabilities_site1 = frequencies/np.sum(frequencies)
axes[0] = plot_bulk_counts(axes[0], bulk_counts, area, lw=4)
axes[0].set_xlabel(None)
axes[0].set_ylabel("P(n) Site 1")

area = 96
bulk_counts_path = root.joinpath(f"POPG_counts_{area}.dat")
bulk_counts = np.loadtxt(bulk_counts_path)
frequencies = np.bincount(bulk_counts.astype(int).flatten())
probabilities_site2 = frequencies/np.sum(frequencies)
axes[1] = plot_bulk_counts(axes[1], bulk_counts, area, lw=4)
axes[1].set_xlabel("n")
axes[1].set_ylabel("P(n) Site 2")

fig.subplots_adjust(left=0.2)
plt.savefig(f"POPG_bulk_distribution_combo.pdf")

In [ ]:
np.sum(probabilities_site2[:4])

In [ ]:
np.sum(probabilities_site1[:2])

# Get Site Distributions

In [ ]:
root = Path("/path/to/data")
replicas = ["rep1", "rep2", "rep3"]
lipid = "POPG"

In [ ]:
site1def = {'theta_start':5, 
            'width':3, 
            'inner_r':24, 
            'outer_r':30, 
            'sitename':"Site 1 ",
            }

site2def = {'theta_start':9,
            'width':4,
            'inner_r':22,
            'outer_r':32,
            'sitename':"Site 2 ",
            }


In [ ]:
leaf = "upp"
# Thresholds
MB =12
AB = 1
EB = 1

# Site definition
the_site_def = site1def

fig, axes = plt.subplots(3,5, figsize=(15,10), sharex=True, sharey=True)
axis = 0
p_unocc = pd.DataFrame(columns=["AB", "MB", "EB"], index=np.arange(0,14))
for rep in replicas:
    data_root = root.joinpath(rep)
    fpath = data_root.joinpath(f"{lipid}.{leaf}.dat")

    num_mol,avg_A,beads,exrho,avg_chain = get_header_info(fpath)
    the_data = np.loadtxt(fpath, skiprows=1)
    rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(fpath)

    try:
        site_1s = make_symmetric_sites(the_data, frames=frames, Ntheta=Ntheta, dr=dr, dth=dth, exrho=1, **the_site_def)
    except:
        print(f"Error on rep {rep}")
        raise
    for site in site_1s:
        site.Npeak = 0
        site.expPunocc = 0.302

    for site, ax, idx in zip(site_1s, axes.flatten()[axis:], np.arange(0,5)):
        p_unocc.at[idx+axis, "AB"] = np.sum(site.densities[:AB])
        p_unocc.at[idx+axis, "MB"] = np.sum(site.densities[:MB])
        p_unocc.at[idx+axis, "EB"] = np.sum(site.densities[:EB])
        ax = plot_density(site, ax)

    axis+=len(site_1s)
 
fig.tight_layout()
plt.savefig(data_root.joinpath(f"site_1s.pdf"))

In [ ]:
print("The average p_unocc for occupancy definition is:")
means = np.average(p_unocc, axis=0)
stdevs = np.std(p_unocc, axis=0)
for label, x, err in zip(p_unocc.columns, means,stdevs):
    print(f"{label}: {np.round(x,1)} ± {np.ceil(err*10)/10}")


In [ ]:
# Plot each replica aggregated
fig, ax = plt.subplots(1,1, figsize=(5,5))
for rep in replicas:
    data_root = root.joinpath(rep)
    fpath = data_root.joinpath(f"{lipid}.{leaf}.dat")

    num_mol,avg_A,beads,exrho,avg_chain = get_header_info(fpath)
    the_data = np.loadtxt(fpath, skiprows=1)
    rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(fpath)

    site_1s = make_symmetric_sites(the_data, frames=frames, Ntheta=Ntheta, dr=dr, dth=dth, exrho=1, **the_site_def)
    for site in site_1s:
        site.Npeak = 0
        site.expPunocc = 0.302

    dummy_site = site_1s[0]
    dummy_site.title=f"average over sites, {lipid}, {leaf}er leaflet, site1 site"
    temp = [site.densities for site in site_1s]
    densities = [np.pad(dens, (0,18-len(dens))) for dens in temp]
    dummy_site.densities = np.average(densities, axis=0)

    ax = plot_density(dummy_site, ax)

plt.savefig(root.joinpath(f"site_1s_averaged_{lipid}.pdf"))